In [1]:
pip install feedparser


  Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
import feedparser
import pandas as pd
from pathlib import Path
import functools
import pathlib
import shutil
import requests
from tqdm.auto import tqdm
import requests
import pickle
DOWNLOAD_FOLDER = Path("data")
FEEDMETA = Path("feedmeta")


def download_episode(url, filename):
    if not filename.exists() and filename.stem not in [x.stem for x in Path("transcriptions").glob("**/*")]:
        r = requests.get(url, stream=True, allow_redirects=True)
        if r.status_code != 200:
            r.raise_for_status()  # Will only raise for 4xx codes, so...
            raise RuntimeError(f"Request to {url} returned status code {r.status_code}")
        file_size = int(r.headers.get('Content-Length', 0))

        path = pathlib.Path(filename).expanduser().resolve()
        path.parent.mkdir(parents=True, exist_ok=True)

        desc = "(Unknown total file size)" if file_size == 0 else ""
        r.raw.read = functools.partial(r.raw.read, decode_content=True)  # Decompress if needed
        with tqdm.wrapattr(r.raw, "read", total=file_size, desc=desc, leave=False) as r_raw:
            with path.open("wb") as f:
                shutil.copyfileobj(r_raw, f)

In [5]:
feeds = [
    # "https://anchor.fm/s/df98870/podcast/rss",
    # "https://feeds.buzzsprout.com/1154870.rss",
    "http://feeds.soundcloud.com/users/soundcloud:users:294272020/sounds.rss",
    "https://feeds.acast.com/public/shows/5eded114811f0313e89ddaeb"
]

for feed in tqdm(feeds, desc="Podcast"):
    podcast = feedparser.parse(feed)
    podcast_title = podcast["feed"]["title"]
    folder_path = DOWNLOAD_FOLDER / podcast_title 
    folder_path.mkdir(exist_ok=True)
    for episode in tqdm(podcast["entries"], desc=f"Podcast {podcast_title[:30]}"):
        download_episode(
            url=[x for x in episode["links"] if x["type"].startswith("audio")][0]["href"],
            filename=folder_path / [x for x in episode["links"] if x["type"].startswith("audio")][0]["href"].split("/")[-1]
        )
        
        
for feed in tqdm(feeds, desc="Podcast"):
    podcast = feedparser.parse(feed)
    podcast_title = podcast["feed"]["title"]
    folder_path = FEEDMETA 
    folder_path.mkdir(exist_ok=True)
    with open(folder_path / f"{podcast_title}.meta.pickle", "wb") as f:
        pickle.dump(podcast, f)

Podcast:   0%|          | 0/2 [00:00<?, ?it/s]

Podcast Farmerama:   0%|          | 0/152 [00:00<?, ?it/s]

Podcast The Regenerative Agroforestry :   0%|          | 0/51 [00:00<?, ?it/s]

Podcast:   0%|          | 0/2 [00:00<?, ?it/s]